In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import inv,eig,det,eigvals
from numpy import log
import allantools
from scipy.signal import welch , periodogram
from scipy.stats import gaussian_kde
%matplotlib qt  

In [2]:
file = pd.read_csv('2018.csv')
data = np.concatenate((np.reshape(file['blueRatio2'].values,(len(file),1)), np.reshape(file['redRatio2'].values,(len(file),1))), axis=1)

up = data[:,0]
down = data[:,1]

shotsPerEllipse = 40
finalEllipse = int(len(data)/shotsPerEllipse)

columns = ['UpInterferometerData','DownInterferometerData','Epoch', 'A', 'B' , 'C' , 'D' , 'E' , 'F' , 'x_center' , 'y_center','Orientation','Phase','x_ellipse','y_ellipse','Sigma','SigmaVariance','MaxLikelihood','Resdual']
FitEllipseData_WhiteAndRandom = pd.DataFrame(columns=columns)
FitEllipseData_WhiteAndAutoRegressive = pd.DataFrame(columns=columns)
FitEllipseData_WhiteAndFliker = pd.DataFrame(columns=columns)
FitEllipseData_White = pd.DataFrame(columns=columns)

In [3]:
plt.figure()
plt.scatter(up[0:100],down[0:100])
plt.grid()

plt.figure()
fig = plt.subplots()
plt.plot(down[0:100],linestyle = '-',color = 'black',marker = '.')
plt.plot(up[0:100],linestyle = '-.',color = 'black')



libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [4]:
epoch = 0

for i in range(0,shotsPerEllipse*finalEllipse,shotsPerEllipse): 
    x = up[0+i:i+shotsPerEllipse]
    y = down[0+i:i+shotsPerEllipse]
    x = np.reshape(x,(len(x),1))
    y = np.reshape(y,(len(y),1))


    A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
    
    S = np.dot(A.T,A)

    C = np.zeros([6,6])
    C[0,2] = C[2,0] = 2; C[1,1] = -1
    S = S.astype(float)
    E, V =  eig(np.dot(inv(S), C))
    n = np.argmax(np.abs(E))
    coff = V[:,n]

    t = np.dot(A,coff)
    t1 = t
    Sk_1 = np.array([[1],[1]])

    df = pd.DataFrame()



    rw_maker = np.linspace(1,len(t),len(t)) 
    ti = np.tile(np.reshape(rw_maker,(len(t),1)), (1, len(t)))
    tj = np.tile(np.reshape(rw_maker,(1,len(t))), (len(t),1))
    Qrx = ti+tj - abs(ti-tj)
    Qry = ti+tj - abs(ti-tj)
    Qwx = np.eye(len(t))
    Qwy = np.eye(len(t))
    Qr = np.zeros((2*len(t),2*len(t)))
    Qr[0:len(t),0:len(t)] = Qrx/2
    Qr[len(t):,len(t):] = Qry/2
    Qw = np.zeros((2*len(t),2*len(t)))
    Qw[0:len(t),0:len(t)] = Qwx
    Qw[len(t):,len(t):] = Qwy



    while 1:
        A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
        t = A@coff
        Bx = np.zeros((len(t),len(t)))
        By = np.zeros((len(t),len(t)))
        np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
        np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
        B = np.concatenate((Bx, By), axis=1)
        
        Qx = Sk_1[0]*Qw + Sk_1[1]*Qr
        Qt = np.dot(np.dot(B,Qx),B.T)

        Sk = Sk_1

        lw = 0.5*t.T@inv(Qt)@B@Qw@B.T@inv(Qt)@t
        lr = 0.5*t.T@inv(Qt)@B@Qr@B.T@inv(Qt)@t
        nwwl = 0.5*np.trace(B@Qw@B.T@inv(Qt)@B@Qw@B.T@inv(Qt))
        nrrl = 0.5*np.trace(B@Qr@B.T@inv(Qt)@B@Qr@B.T@inv(Qt))
        nrwl = 0.5*np.trace(B@Qr@B.T@inv(Qt)@B@Qw@B.T@inv(Qt))
        nwrl = 0.5*np.trace(B@Qw@B.T@inv(Qt)@B@Qr@B.T@inv(Qt))

        N = np.array([[nwwl,nwrl],[nrwl,nrrl]])
        l = np.array([[lw],[lr]])

        Sk_1 = inv(N)@l
        diff_Sk = Sk - Sk_1
        if diff_Sk.T@diff_Sk <10**(-4):
            A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
            t = A@coff
            Bx = np.zeros((len(t),len(t)))
            By = np.zeros((len(t),len(t)))
            np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
            np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
            B = np.concatenate((Bx, By), axis=1)
            Qx = Sk_1[0]*Qw + Sk_1[1]*Qr
            Qt = np.dot(np.dot(B,Qx),B.T)
            coff = inv(A.T@inv(Qt)@A)@A.T@inv(Qt)@t

            break


    b = np.size(N, 0)
    L = -0.5*np.sum(log(np.abs(eigvals(Qt))))-0.5*(t.T@inv(Qt)@t) - b/2*log(2*np.pi)

    si = np.linspace(0, 2*np.pi, 100)

    P2,P3,P4,P5,P6,P1 = coff[1], coff[2], coff[3], coff[4], coff[5], coff[0] # P1x^2 + P2xy + P3y^2 + P4x + P5y + P6 = 0

    x_c = (P3*P4/2 - P2*P5/4)/(P2**2/4 - P1*P3)
    y_c = (P1*P5/2 - P2*P4/4)/(P2**2/4 - P1*P3)
    theta = 0.5*np.arctan(P2/(P1-P3))


    SemiMajor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P3-P1)*np.sqrt(1+P2**2/(P1-P3)**2) - (P3+P1))))
    SemiMinor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P1-P3)*np.sqrt(1+P2**2/(P1-P3)**2) - (P1+P3))))


    phase_gg = np.arccos(-P2/(2*np.sqrt(P1*P3)))

        # Parametric equations for the rotated ellipse
    x_rotated = x_c + SemiMajor_Ax * np.cos(si) * np.cos(theta) - SemiMinor_Ax * np.sin(si) * np.sin(theta)
    y_rotated = y_c + SemiMajor_Ax * np.cos(si) * np.sin(theta) + SemiMinor_Ax * np.sin(si) * np.cos(theta)

    epoch = epoch+1
    # ['UpInterferometerData','DownInterferometerData','Epoch', 'A', 'B' , 'C' , 'D' , 'E' , 'F' , 'x_center' , 'y_center','Orientation','Phase','x_ellipse','y_ellipse','Sigma','SigmaVariance']
    infor = [{'UpInterferometerData':x,'DownInterferometerData':y,'Epoch':epoch,'A':coff[0],'B':coff[1],'C':coff[2],'D':coff[3],'E':coff[4],'F':coff[5],'x_center':x_c,'y_center':y_c,'Orientation':theta,'Phase':phase_gg,'x_ellipse':x_rotated,'y_ellipse':y_rotated,'Sigma':Sk_1,'SigmaVariance':inv(N),'MaxLikelihood':L,'Residual':t}]
    FitEllipseData_WhiteAndRandom = pd.concat([FitEllipseData_WhiteAndRandom, pd.DataFrame(infor, index=[0])], ignore_index=True)
    


In [5]:
import seaborn as sns

plt.figure()
plt.plot(FitEllipseData_WhiteAndRandom['x_ellipse'].iloc[1],FitEllipseData_WhiteAndRandom['y_ellipse'].iloc[1],color = 'black')
plt.scatter(FitEllipseData_WhiteAndRandom['UpInterferometerData'].iloc[1],FitEllipseData_WhiteAndRandom['DownInterferometerData'].iloc[1],color = 'black')
plt.gca().set_facecolor('white')
plt.grid()


In [6]:
epoch = 0

for i in range(0,shotsPerEllipse*finalEllipse,shotsPerEllipse): 
    x = up[0+i:i+shotsPerEllipse]
    y = down[0+i:i+shotsPerEllipse]
    x = np.reshape(x,(len(x),1))
    y = np.reshape(y,(len(y),1))


    A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
    
    S = np.dot(A.T,A)

    C = np.zeros([6,6])
    C[0,2] = C[2,0] = 2; C[1,1] = -1
    S = S.astype(float)
    E, V =  eig(np.dot(inv(S), C))
    n = np.argmax(np.abs(E))
    coff = V[:,n]

    t = np.dot(A,coff)
    t1 = t
    Sk_1 = np.array([[1],[1]])

    df = pd.DataFrame()



    m = len(t)  # Assuming m is some integer value

    t = np.arange(1, m+1)[:, np.newaxis]
    tau = np.abs(np.tile(t, (1, m)) - np.tile(t.T, (m, 1)))
    Qam = np.exp(-tau)
    Qwx = np.eye(len(t))
    Qwy = np.eye(len(t))
    Qatr = np.zeros((2*len(t),2*len(t)))
    Qatr[0:len(t),0:len(t)] = Qam
    Qatr[len(t):,len(t):] = Qam
    Qw = np.zeros((2*len(t),2*len(t)))
    Qw[0:len(t),0:len(t)] = Qwx
    Qw[len(t):,len(t):] = Qwy

  

    while 1:
        A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
        t = A@coff
        Bx = np.zeros((len(t),len(t)))
        By = np.zeros((len(t),len(t)))
        np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
        np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
        B = np.concatenate((Bx, By), axis=1)
        
        Qx = Sk_1[0]*Qw + Sk_1[1]*Qatr
        Qt = np.dot(np.dot(B,Qx),B.T)

        coff = inv(A.T@inv(Qt)@A)@A.T@inv(Qt)@t
        
        Sk = Sk_1

        lw = 0.5*t.T@inv(Qt)@B@Qw@B.T@inv(Qt)@t
        la = 0.5*t.T@inv(Qt)@B@Qatr@B.T@inv(Qt)@t
        nwwl = 0.5*np.trace(B@Qw@B.T@inv(Qt)@B@Qw@B.T@inv(Qt))
        naal = 0.5*np.trace(B@Qatr@B.T@inv(Qt)@B@Qatr@B.T@inv(Qt))
        nawl = 0.5*np.trace(B@Qatr@B.T@inv(Qt)@B@Qw@B.T@inv(Qt))
        nwal = 0.5*np.trace(B@Qw@B.T@inv(Qt)@B@Qatr@B.T@inv(Qt))

        N = np.array([[nwwl,nawl],[nwal,naal]])
        l = np.array([[lw],[la]])

        Sk_1 = inv(N)@l
        diff_Sk = Sk - Sk_1
        if diff_Sk.T@diff_Sk <10**(-4):
            A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
            t = A@coff
            Bx = np.zeros((len(t),len(t)))
            By = np.zeros((len(t),len(t)))
            np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
            np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
            B = np.concatenate((Bx, By), axis=1)
            Qx = Sk_1[0]*Qw + Sk_1[1]*Qatr
            Qt = np.dot(np.dot(B,Qx),B.T)
            coff = inv(A.T@inv(Qt)@A)@A.T@inv(Qt)@t

            break


    b = np.size(N, 0)
    L = -0.5*np.sum(log(np.abs(eigvals(Qt))))-0.5*(t.T@inv(Qt)@t) - b/2*log(2*np.pi)

    si = np.linspace(0, 2*np.pi, 100)

    P2,P3,P4,P5,P6,P1 = coff[1], coff[2], coff[3], coff[4], coff[5], coff[0] # P1x^2 + P2xy + P3y^2 + P4x + P5y + P6 = 0

    x_c = (P3*P4/2 - P2*P5/4)/(P2**2/4 - P1*P3)
    y_c = (P1*P5/2 - P2*P4/4)/(P2**2/4 - P1*P3)
    theta = 0.5*np.arctan(P2/(P1-P3))


    SemiMajor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P3-P1)*np.sqrt(1+P2**2/(P1-P3)**2) - (P3+P1))))
    SemiMinor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P1-P3)*np.sqrt(1+P2**2/(P1-P3)**2) - (P1+P3))))


    phase_gg = np.arccos(-P2/(2*np.sqrt(P1*P3)))

        # Parametric equations for the rotated ellipse
    x_rotated = x_c + SemiMajor_Ax * np.cos(si) * np.cos(theta) - SemiMinor_Ax * np.sin(si) * np.sin(theta)
    y_rotated = y_c + SemiMajor_Ax * np.cos(si) * np.sin(theta) + SemiMinor_Ax * np.sin(si) * np.cos(theta)

    epoch = epoch+1
    # ['UpInterferometerData','DownInterferometerData','Epoch', 'A', 'B' , 'C' , 'D' , 'E' , 'F' , 'x_center' , 'y_center','Orientation','Phase','x_ellipse','y_ellipse','Sigma','SigmaVariance']
    infor = [{'UpInterferometerData':x,'DownInterferometerData':y,'Epoch':epoch,'A':coff[0],'B':coff[1],'C':coff[2],'D':coff[3],'E':coff[4],'F':coff[5],'x_center':x_c,'y_center':y_c,'Orientation':theta,'Phase':phase_gg,'x_ellipse':x_rotated,'y_ellipse':y_rotated,'Sigma':Sk_1,'SigmaVariance':inv(N),'MaxLikelihood':L,'Residual':t}]
    FitEllipseData_WhiteAndAutoRegressive = pd.concat([FitEllipseData_WhiteAndAutoRegressive, pd.DataFrame(infor, index=[0])], ignore_index=True)
    


In [7]:
epoch = 0

for i in range(0,shotsPerEllipse*finalEllipse,shotsPerEllipse): 
    x = up[0+i:i+shotsPerEllipse]
    y = down[0+i:i+shotsPerEllipse]
    x = np.reshape(x,(len(x),1))
    y = np.reshape(y,(len(y),1))


    A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
    
    S = np.dot(A.T,A)

    C = np.zeros([6,6])
    C[0,2] = C[2,0] = 2; C[1,1] = -1
    S = S.astype(float)
    E, V =  eig(np.dot(inv(S), C))
    n = np.argmax(np.abs(E))
    coff = V[:,n]

    t = np.dot(A,coff)
    t1 = t
    Sk_1 = np.array([[1],[1]])

    df = pd.DataFrame()



    m = len(t)  # Assuming m is some integer value

    t = np.arange(1, m+1)[:, np.newaxis]
    tau = np.abs(np.tile(t, (1, m)) - np.tile(t.T, (m, 1)))
    tau = tau + 0.25 * np.eye(m)
    Qfm = (1 - (((np.log(tau) / np.log(2)) + 2) / 24)) * (9 / 8)
    Qwx = np.eye(len(t))
    Qwy = np.eye(len(t))
    Qfn = np.zeros((2*len(t),2*len(t)))
    Qfn[0:len(t),0:len(t)] = Qfm
    Qfn[len(t):,len(t):] = Qfm
    Qw = np.zeros((2*len(t),2*len(t)))
    Qw[0:len(t),0:len(t)] = Qwx
    Qw[len(t):,len(t):] = Qwy

  

    while 1:
        A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
        t = A@coff
        Bx = np.zeros((len(t),len(t)))
        By = np.zeros((len(t),len(t)))
        np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
        np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
        B = np.concatenate((Bx, By), axis=1)
        
        Qx = Sk_1[0]*Qw + Sk_1[1]*Qfn
        Qt = np.dot(np.dot(B,Qx),B.T)

        coff = inv(A.T@inv(Qt)@A)@A.T@inv(Qt)@t
        
        Sk = Sk_1

        lw = 0.5*t.T@inv(Qt)@B@Qw@B.T@inv(Qt)@t
        lf = 0.5*t.T@inv(Qt)@B@Qfn@B.T@inv(Qt)@t
        nwwl = 0.5*np.trace(B@Qw@B.T@inv(Qt)@B@Qw@B.T@inv(Qt))
        nffl = 0.5*np.trace(B@Qfn@B.T@inv(Qt)@B@Qfn@B.T@inv(Qt))
        nfwl = 0.5*np.trace(B@Qfn@B.T@inv(Qt)@B@Qw@B.T@inv(Qt))
        nwfl = 0.5*np.trace(B@Qw@B.T@inv(Qt)@B@Qfn@B.T@inv(Qt))

        N = np.array([[nwwl,nfwl],[nwfl,nffl]])
        l = np.array([[lw],[lf]])

        Sk_1 = inv(N)@l
        diff_Sk = Sk - Sk_1
        if diff_Sk.T@diff_Sk <10**(-4):
            A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
            t = A@coff
            Bx = np.zeros((len(t),len(t)))
            By = np.zeros((len(t),len(t)))
            np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
            np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
            B = np.concatenate((Bx, By), axis=1)
            Qx = Sk_1[0]*Qw + Sk_1[1]*Qfn
            Qt = np.dot(np.dot(B,Qx),B.T)
            coff = inv(A.T@inv(Qt)@A)@A.T@inv(Qt)@t

            break


    b = np.size(N, 0)
    L = -0.5*np.sum(log(np.abs(eigvals(Qt))))-0.5*(t.T@inv(Qt)@t) - b/2*log(2*np.pi)

    si = np.linspace(0, 2*np.pi, 100)

    P2,P3,P4,P5,P6,P1 = coff[1], coff[2], coff[3], coff[4], coff[5], coff[0] # P1x^2 + P2xy + P3y^2 + P4x + P5y + P6 = 0

    x_c = (P3*P4/2 - P2*P5/4)/(P2**2/4 - P1*P3)
    y_c = (P1*P5/2 - P2*P4/4)/(P2**2/4 - P1*P3)
    theta = 0.5*np.arctan(P2/(P1-P3))


    SemiMajor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P3-P1)*np.sqrt(1+P2**2/(P1-P3)**2) - (P3+P1))))
    SemiMinor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P1-P3)*np.sqrt(1+P2**2/(P1-P3)**2) - (P1+P3))))


    phase_gg = np.arccos(-P2/(2*np.sqrt(P1*P3)))

        # Parametric equations for the rotated ellipse
    x_rotated = x_c + SemiMajor_Ax * np.cos(si) * np.cos(theta) - SemiMinor_Ax * np.sin(si) * np.sin(theta)
    y_rotated = y_c + SemiMajor_Ax * np.cos(si) * np.sin(theta) + SemiMinor_Ax * np.sin(si) * np.cos(theta)

    epoch = epoch+1
    # ['UpInterferometerData','DownInterferometerData','Epoch', 'A', 'B' , 'C' , 'D' , 'E' , 'F' , 'x_center' , 'y_center','Orientation','Phase','x_ellipse','y_ellipse','Sigma','SigmaVariance']
    infor = [{'UpInterferometerData':x,'DownInterferometerData':y,'Epoch':epoch,'A':coff[0],'B':coff[1],'C':coff[2],'D':coff[3],'E':coff[4],'F':coff[5],'x_center':x_c,'y_center':y_c,'Orientation':theta,'Phase':phase_gg,'x_ellipse':x_rotated,'y_ellipse':y_rotated,'Sigma':Sk_1,'SigmaVariance':inv(N),'MaxLikelihood':L,'Residual':t}]
    FitEllipseData_WhiteAndFliker = pd.concat([FitEllipseData_WhiteAndFliker, pd.DataFrame(infor, index=[0])], ignore_index=True)
    


In [8]:
epoch = 0

for i in range(0,shotsPerEllipse*finalEllipse,shotsPerEllipse): 
    x = up[0+i:i+shotsPerEllipse]
    y = down[0+i:i+shotsPerEllipse]
    x = np.reshape(x,(len(x),1))
    y = np.reshape(y,(len(y),1))


    A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
    
    S = np.dot(A.T,A)

    C = np.zeros([6,6])
    C[0,2] = C[2,0] = 2; C[1,1] = -1
    S = S.astype(float)
    E, V =  eig(np.dot(inv(S), C))
    n = np.argmax(np.abs(E))
    coff = V[:,n]

    t = np.dot(A,coff)
    t1 = t
    Sk_1 = np.array([[1]])

    df = pd.DataFrame()



    m = len(t)  # Assuming m is some integer value


    Qwx = np.eye(len(t))
    Qwy = np.eye(len(t))
    Qw = np.zeros((2*len(t),2*len(t)))
    Qw[0:len(t),0:len(t)] = Qwx
    Qw[len(t):,len(t):] = Qwy

  

    while 1:
        A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
        t = A@coff
        Bx = np.zeros((len(t),len(t)))
        By = np.zeros((len(t),len(t)))
        np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
        np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
        B = np.concatenate((Bx, By), axis=1)
        
        Qx = Sk_1[0]*Qw 
        Qt = np.dot(np.dot(B,Qx),B.T)

        coff = inv(A.T@inv(Qt)@A)@A.T@inv(Qt)@t
        
        Sk = Sk_1

        lw = 0.5*t.T@inv(Qt)@B@Qw@B.T@inv(Qt)@t
 
        nwwl = 0.5*np.trace(B@Qw@B.T@inv(Qt)@B@Qw@B.T@inv(Qt))


        N = np.array([[nwwl]])
        l = np.array([[lw]])

        Sk_1 = inv(N)@l
        diff_Sk = Sk - Sk_1
        if diff_Sk.T@diff_Sk <10**(-4):
            A =  np.hstack((x*x, x*y, y*y, x, y, np.ones_like(x)))
            t = A@coff
            Bx = np.zeros((len(t),len(t)))
            By = np.zeros((len(t),len(t)))
            np.fill_diagonal(Bx, 2*coff[0]*x + coff[1]*y + coff[3])
            np.fill_diagonal(By, 2*coff[3]*y + coff[1]*x + coff[4])
            B = np.concatenate((Bx, By), axis=1)
            Qx = Sk_1[0]*Qw
            Qt = np.dot(np.dot(B,Qx),B.T)
            coff = inv(A.T@inv(Qt)@A)@A.T@inv(Qt)@t

            break


    b = np.size(N, 0)
    L = -0.5*np.sum(log(np.abs(eigvals(Qt))))-0.5*(t.T@inv(Qt)@t) - b/2*log(2*np.pi)

    si = np.linspace(0, 2*np.pi, 100)

    P2,P3,P4,P5,P6,P1 = coff[1], coff[2], coff[3], coff[4], coff[5], coff[0] # P1x^2 + P2xy + P3y^2 + P4x + P5y + P6 = 0

    x_c = (P3*P4/2 - P2*P5/4)/(P2**2/4 - P1*P3)
    y_c = (P1*P5/2 - P2*P4/4)/(P2**2/4 - P1*P3)
    theta = 0.5*np.arctan(P2/(P1-P3))


    SemiMajor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P3-P1)*np.sqrt(1+P2**2/(P1-P3)**2) - (P3+P1))))
    SemiMinor_Ax = np.sqrt(2*(P1*P5**2/4 + P3*P4**2/4 + P6*P2**2/4 - P2*P4*P5/4 -P1*P3*P6)/((P2**2/4 - P1*P3)*( (P1-P3)*np.sqrt(1+P2**2/(P1-P3)**2) - (P1+P3))))


    phase_gg = np.arccos(-P2/(2*np.sqrt(P1*P3)))

        # Parametric equations for the rotated ellipse
    x_rotated = x_c + SemiMajor_Ax * np.cos(si) * np.cos(theta) - SemiMinor_Ax * np.sin(si) * np.sin(theta)
    y_rotated = y_c + SemiMajor_Ax * np.cos(si) * np.sin(theta) + SemiMinor_Ax * np.sin(si) * np.cos(theta)

    epoch = epoch+1
    # ['UpInterferometerData','DownInterferometerData','Epoch', 'A', 'B' , 'C' , 'D' , 'E' , 'F' , 'x_center' , 'y_center','Orientation','Phase','x_ellipse','y_ellipse','Sigma','SigmaVariance']
    infor = [{'UpInterferometerData':x,'DownInterferometerData':y,'Epoch':epoch,'A':coff[0],'B':coff[1],'C':coff[2],'D':coff[3],'E':coff[4],'F':coff[5],'x_center':x_c,'y_center':y_c,'Orientation':theta,'Phase':phase_gg,'x_ellipse':x_rotated,'y_ellipse':y_rotated,'Sigma':Sk_1,'SigmaVariance':inv(N),'MaxLikelihood':L,'Residual':t}]
    FitEllipseData_White = pd.concat([FitEllipseData_White, pd.DataFrame(infor, index=[0])], ignore_index=True)
    


In [9]:
taus , adevs, errors, ns = allantools.oadev(data[:,0])
adevs = adevs*2
plt.figure(figsize=(8, 6))
plt.subplot(2,2,1)
plt.loglog(taus,adevs,color = 'black')
plt.xlabel('Tau (s)', fontsize=11, fontweight='bold')
plt.ylabel(r'$\sigma^2_{down}$', labelpad=15, fontsize=11, fontweight='bold')
plt.grid('True',which='minor')

d = np.zeros((len(taus)-1,1))
for i in range(0,len(taus)-1):
    d[i,0] = (np.log(adevs[i+1])-np.log(adevs[i]))/(np.log(taus[i+1])-np.log(taus[i]))
    

plt.subplot(2,2,2)
plt.plot(d,'o',color = 'black')
plt.plot(np.linspace(0,len(d),len(d)),np.mean(d)*np.ones((len(d),1)),color = 'black')
plt.xlabel('Slope Number', fontsize=11, fontweight='bold')
plt.ylabel('Slope of lower garvimeter', fontsize=11, fontweight='bold')
plt.ylim([-1-0.1,-1+0.1])
plt.grid()
plt.gca().yaxis.set_ticks_position('right')
plt.gca().yaxis.set_label_position('right')
# plt.gca().spines['left'].set_visible(False)

taus , adevs, errors, ns = allantools.oadev(data[:,1])
adevs = adevs*2
plt.subplot(223)
plt.loglog(taus,adevs,color = 'black')
plt.xlabel('Tau (s)', fontsize=11, fontweight='bold')
plt.ylabel(r'$\sigma^2_{down}$', labelpad=15, fontsize=11, fontweight='bold')
plt.grid('True',which='minor')

d = np.zeros((len(taus)-1,1))
for i in range(0,len(taus)-1):
    d[i,0] = (np.log(adevs[i+1])-np.log(adevs[i]))/(np.log(taus[i+1])-np.log(taus[i]))
    

plt.subplot(2,2,4)
plt.plot(d,'o',color = 'black')
plt.plot(np.linspace(0,len(d),len(d)),np.mean(d)*np.ones((len(d),1)),color = 'black')
plt.xlabel('Slope Number', fontsize=11, fontweight='bold')
plt.ylabel('Slope of upper garvimeter', fontsize=11, fontweight='bold')
plt.ylim([-1-0.1,-1+0.1])
plt.grid()
plt.gca().yaxis.set_ticks_position('right')
plt.gca().yaxis.set_label_position('right')
# plt.gca().spines['left'].set_visible(False)


plt.suptitle('Alllan Variance DataSet2', fontsize=14, fontweight='bold')
plt.tight_layout()

plt.savefig('/home/kali/Desktop/Article/OutPut/figure4.png', dpi=800)


In [10]:
# Enable interactive plots
# %matplotlib qt  

# Compute the power spectral density using Welch's method
frequencies, psd = welch(data[:,0])

# Plot the power spectral density
plt.figure(figsize=(6, 4))
plt.semilogy(frequencies[3:len(psd)-3], psd[3:len(psd)-3], label = 'Lower garvimeter',color = 'black')
plt.xlabel('Frequency (Hz)', fontsize=11, fontweight='bold')
plt.ylabel('Power/Frequency (dB/Hz)', fontsize=11, fontweight='bold')
plt.title('Power Spectral Density', fontsize=11, fontweight='bold')
plt.grid(True)

frequencies, psd = welch(data[:,1])
plt.semilogy(frequencies[3:len(psd)-3], psd[3:len(psd)-3],label = 'Upper garvimeter',color = 'gray')
plt.legend()
plt.tight_layout()

plt.savefig('/home/kali/Desktop/Article/OutPut/figure8.png', dpi=400)


In [11]:
fs = 100  # Sampling frequency in Hz

# Compute the Power Spectral Density (PSD) using periodogram method
frequencies, Pxx = periodogram(data[:,0], fs=fs)

# Plot the Power Spectral Density
plt.figure()
plt.plot(frequencies[3:len(psd)-3], 10 * np.log10(Pxx[3:len(psd)-3]))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power/Frequency (dB/Hz)')
plt.title('Power Spectral Density (PSD)')
plt.grid(True)
plt.show()

In [12]:
FitEllipseData_WhiteAndRandom['MaxLikelihood']

0      277.740055
1      287.053049
2      275.740167
3      287.145554
4      279.870087
          ...    
666    280.718148
667    282.570767
668    280.846068
669    288.069682
670    285.459435
Name: MaxLikelihood, Length: 671, dtype: object

In [13]:
FitEllipseData_White['MaxLikelihood']

0      278.907653
1      288.126501
2      277.760942
3      288.996927
4      279.932228
          ...    
666    281.451815
667    282.945456
668    281.417166
669    287.539393
670    284.246633
Name: MaxLikelihood, Length: 671, dtype: object

In [14]:
FitEllipseData_WhiteAndFliker['MaxLikelihood']

0      277.305256
1      286.619393
2      275.926841
3      287.690956
4      278.709636
          ...    
666    279.844782
667    281.130176
668    281.296177
669    285.879491
670    283.883351
Name: MaxLikelihood, Length: 671, dtype: object

In [15]:
FitEllipseData_WhiteAndAutoRegressive['MaxLikelihood']

0      278.383007
1      287.728391
2      277.672485
3      288.337422
4      279.519109
          ...    
666     281.01567
667    274.521316
668    280.519538
669    287.904461
670     283.32778
Name: MaxLikelihood, Length: 671, dtype: object

In [16]:
print(FitEllipseData_WhiteAndRandom['MaxLikelihood'].mean())
print(FitEllipseData_White['MaxLikelihood'].mean())
print(FitEllipseData_WhiteAndFliker['MaxLikelihood'].mean())
print(FitEllipseData_WhiteAndAutoRegressive['MaxLikelihood'].mean())

MaxLikelihood = np.array([[FitEllipseData_WhiteAndRandom['MaxLikelihood'].mean()],[FitEllipseData_White['MaxLikelihood'].mean()]
                          ,[FitEllipseData_WhiteAndFliker['MaxLikelihood'].mean()],[FitEllipseData_WhiteAndAutoRegressive['MaxLikelihood'].mean()]])
np.savetxt('Dataset2_Maximumliklehood.txt', MaxLikelihood)

280.85512638843693
280.6876093516039
279.8218609423106
280.59458075354246


In [17]:
resd = np.stack(FitEllipseData_White['Residual'])
m,n = np.shape(resd)
resd = np.reshape(resd,(m*n,1))
np.savetxt('resd2.txt', resd)

In [18]:
SigmaWhite = np.stack(FitEllipseData_White['Sigma'])
meanSigmaWhite = SigmaWhite.mean()
print(meanSigmaWhite)

SigmaVarianceWhite = np.stack(FitEllipseData_White['SigmaVariance'])
meanSigmaVarianceWhite = SigmaVarianceWhite.mean()/100
print(meanSigmaVarianceWhite)

5.664168561387437e-07
1.7175993254031012e-16


In [19]:
SigmaWhiteAndRandom = np.stack(FitEllipseData_WhiteAndRandom['Sigma'])
m,n,p = np.shape(SigmaWhiteAndRandom)
SigmaWhiteAndRandom = SigmaWhiteAndRandom.reshape(m, 2)
meanSigmaWhiteAndRandom_white = SigmaWhiteAndRandom[:,0].mean()
meanSigmaWhiteAndRandom_Random = SigmaWhiteAndRandom[:,1].mean()
print(meanSigmaWhiteAndRandom_white,meanSigmaWhiteAndRandom_Random)

SigmaVarianceWhiteAndRandom = np.stack(FitEllipseData_WhiteAndRandom['SigmaVariance'])
m,n,p = np.shape(SigmaVarianceWhiteAndRandom)
SigmaVarianceWhiteAndRandom = SigmaVarianceWhiteAndRandom.reshape(m, 4)
meanSigmaVarianceWhiteAndRandom_white = SigmaVarianceWhiteAndRandom[:,0].mean()/100
meanSigmaVarianceWhiteAndRandom_Random = SigmaVarianceWhiteAndRandom[:,3].mean()/10000
print(meanSigmaVarianceWhiteAndRandom_white,meanSigmaVarianceWhiteAndRandom_Random)


5.686317982789153e-07 -4.119955032395149e-09
2.3323826769266817e-16 1.256091170007142e-19


In [20]:
SigmaWhiteAndFliker = np.stack(FitEllipseData_WhiteAndFliker['Sigma'])
m,n,p = np.shape(SigmaWhiteAndFliker)
SigmaWhiteAndFliker = SigmaWhiteAndFliker.reshape(m, 2)
meanSigmaWhiteAndFliker_white = SigmaWhiteAndFliker[:,0].mean()
meanSigmaWhiteAndFliker_Fliker = SigmaWhiteAndFliker[:,1].mean()
print(meanSigmaWhiteAndFliker_white,meanSigmaWhiteAndFliker_Fliker)

SigmaVarianceWhiteAndFliker = np.stack(FitEllipseData_WhiteAndFliker['SigmaVariance'])
m,n,p = np.shape(SigmaVarianceWhiteAndFliker)
SigmaVarianceWhiteAndFliker = SigmaVarianceWhiteAndFliker.reshape(m, 4)
meanSigmaVarianceWhiteAndFliker_white = SigmaVarianceWhiteAndFliker[:,0].mean()/100
meanSigmaVarianceWhiteAndFliker_Fliker = SigmaVarianceWhiteAndFliker[:,3].mean()/10000
print(meanSigmaVarianceWhiteAndFliker_white,meanSigmaVarianceWhiteAndFliker_Fliker)

5.753775189292566e-07 -3.6518473670782165e-08
2.196934810025396e-16 3.126961214583308e-18


In [21]:
SigmaWhiteAndAutoRegressive = np.stack(FitEllipseData_WhiteAndAutoRegressive['Sigma'])
m,n,p = np.shape(SigmaWhiteAndAutoRegressive)
SigmaWhiteAndAutoRegressive = SigmaWhiteAndAutoRegressive.reshape(m, 2)
meanSigmaWhiteAndAutoRegressive_white = SigmaWhiteAndAutoRegressive[:,0].mean()
meanSigmaWhiteAndAutoRegressive_AutoRegressive = SigmaWhiteAndAutoRegressive[:,1].mean()
print(meanSigmaWhiteAndAutoRegressive_white,meanSigmaWhiteAndAutoRegressive_AutoRegressive)

SigmaVarianceWhiteAndAutoRegressive = np.stack(FitEllipseData_WhiteAndAutoRegressive['SigmaVariance'])
m,n,p = np.shape(SigmaVarianceWhiteAndAutoRegressive)
SigmaVarianceWhiteAndAutoRegressive = SigmaVarianceWhiteAndAutoRegressive.reshape(m, 4)
meanSigmaVarianceWhiteAndFliker_white = SigmaVarianceWhiteAndAutoRegressive[:,0].mean()/100
meanSigmaVarianceWhiteAndFliker_AutoRegressive = SigmaVarianceWhiteAndAutoRegressive[:,3].mean()/10000
print(meanSigmaVarianceWhiteAndFliker_white,meanSigmaVarianceWhiteAndFliker_AutoRegressive)

6.235577983003819e-07 -5.4879982113221643e-08
7.405090062145583e-16 5.264621209767184e-18


In [22]:
A = np.stack(FitEllipseData_White['A'])
B = np.stack(FitEllipseData_White['B'])
C = np.stack(FitEllipseData_White['C'])
phi = np.arccos(A/(2*np.sqrt(B*C)))
np.stack(FitEllipseData_White['Phase'])

/tmp/ipykernel_4364/1801834442.py:4: RuntimeWarning: invalid value encountered in sqrt
  phi = np.arccos(A/(2*np.sqrt(B*C)))


array([2.26704072, 2.28688428, 2.1973994 , 2.24858897, 2.28063379,
       2.25454331, 2.24183474, 2.22635121, 2.25910709, 2.2494602 ,
       2.23210291, 2.24025213, 2.2298927 , 2.24549773, 2.25680115,
       2.22323498, 2.22427396, 2.22702484, 2.2355397 , 2.2375053 ,
       2.2665763 , 2.23690457, 2.24118213, 2.20981587, 2.28505138,
       2.24176153, 2.27366218, 2.25326997, 2.21349246, 0.89704702,
       2.24433429, 0.91440799, 2.26681581, 2.24467707, 2.23938147,
       2.27204332, 2.27276732, 2.24526143, 2.22001191, 2.23414273,
       2.22580695, 2.26385187, 0.88410408, 2.25427386, 2.25187483,
       2.25729087, 0.88258512, 0.84337285, 2.22233598, 2.2252474 ,
       2.22847602, 2.25875162, 2.25336823, 2.27384795, 2.23529211,
       2.25961171, 2.25131672, 2.23375689, 2.25386356, 2.22629903,
       2.22256166, 2.26160129, 2.2864106 , 2.24413595, 2.20536271,
       2.23861805, 2.21564257, 2.28572563, 2.2561881 , 2.2392808 ,
       2.25191478, 2.25675969, 2.23854875, 2.2576032 , 2.27398